# Milestone Project 1: Food Vision Big

In [ ]:
!nvidia-smi -L

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py

In [ ]:
from helper_functions import create_tensorboard_callback,plot_loss_curves,compare_historys

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
(train_data,test_data),ds_info = tfds.load(name="food101",
                                          split=["train","validation"],
                                            shuffle_files=True,
                                           as_supervised=True,
                                           with_info=True
                                          )

In [ ]:
ds_info.features

In [ ]:
class_names = ds_info.features["label"].names

In [ ]:
# Take on sample of train data
train_one_sample = train_data.take(1)

In [ ]:
train_one_sample

In [ ]:
for image,label in train_one_sample:
    print(f"""
    Image shapes: {image.shape}
    Image datatype: {image.dtype}
    Target class from Food101 (tensor form): {label}
    Class name (str form): {class_names[label.numpy()]}
    """)

In [ ]:
image

In [ ]:
import tensorflow as tf
tf.reduce_min(image),tf.reduce_max(image)

In [ ]:
# plot an image 
import matplotlib.pyplot as plt
plt.figure(figsize=(10,7))
plt.imshow(image)
plt.title(class_names[label.numpy()])
plt.axis(False)

In [ ]:
def preprocess_img(image,label,img_shape=224):
    """
    converts image datatype form 'uint8' -> 'float32' and reshpaes
    image to [img_shape,img_shape,color_channel]
    """
    image = tf.image.resize(image,[img_shpae,img_shape])
    return tf.cast(image,tf.float32),label

In [ ]:
preprocessed_image = preprocess_img(image,label)[0]
print(f"Image before preprocessing:\n {image[:2]}.. \nshape:{image.shape},\ndatatype: {image.dtype}")
print(f"iamge after preprocessing:\n{preprocessed_image[:2]}...\nshape:{preprocessed_image.shape}\ndatatype:{preprocessed_image.dtype} ")